In [1]:
using CairoMakie, LinearAlgebra, Dierckx, Optim

In [11]:
pars = (;α = 0.33, # Capital share
         β = 0.9, # Discount factor
         γ = 1.0, # CRRA parameter
         δ = 0.1, # Depreciation rate
         A = 2.0, # TFP
         θ = 3.0, # Expanding grid parameter
         grid_size = 101, # number of gridpoints
         grid_min = 0.75, # minimum gridpoint (multiple of steady state capital stock)
         grid_max = 1.25, # maximum gridpoint (multiple of steady state capital stock)
         maxiter = 10000, # maximum number of iterations
         toler = 1e-6) # tolerance

(α = 0.33, β = 0.9, γ = 1.0, δ = 0.1, A = 2.0, θ = 3.0, grid_size = 101, grid_min = 0.75, grid_max = 1.25, maxiter = 10000, toler = 1.0e-6)

In [12]:
function utility(c, pars)
    if pars.γ == 1
        return log(c)
    else
        return (c^(1-pars.γ))/(1-pars.γ)
    end
end

function utility_vec(c, pars)
    if pars.γ == 1
        return log.(c)
    else
        return (c.^(1-pars.γ))./(1-pars.γ)
    end
end

utility_vec (generic function with 1 method)

In [14]:
function exp_grid(pars)
    (; θ, grid_min, grid_max, grid_size, A, α, β, δ) = pars
    Kss = ((A*α*β)/(1-β*(1.0-δ)))^(1/(1.0-α)) 
    grid = LinRange(grid_min*Kss, grid_max*Kss, grid_size)
    grid = grid.^θ
    return grid
end

exp_grid (generic function with 1 method)